In [1]:
# Importing Modules

import time, math, random
from multipledispatch import dispatch

In [2]:
# Colours

normal = "\033[0;37;48m"
yellow = "\033[0;33;48m"

orange = "\033[0;34;40m"
green = "\033[0;32;40m"
blue = "\033[0;35;40m"

In [3]:
# Files for importing/exporting

file_ID_storage = "ID Storage.txt"
file_habits_info = "Habits Info.txt"
file_logs = "Logs.txt"

In [4]:
# Global Variables

weight_limit = 5

colour_classifier = "\033"
colour_length = len(normal)
time_colour = yellow

display_indentation = "   "
days = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]

ID_length = 3

In [5]:
def remove_colour(message: str) -> str:
    uncoloured_message = []

    for word in message.split(" "):
        if colour_classifier in word:
            classifier_index = word.index(colour_classifier)
            word = word[:classifier_index] + word[classifier_index + colour_length:]

        uncoloured_message.append(word)

    return " ".join(uncoloured_message)

def get_stylized_time() -> str:
    current_time = time.strftime("%I:%M:%S %p  %d-%b-%Y")
    return f"{time_colour}{current_time}{normal}"

In [6]:
class Common_Functions:
    def __init__(self) -> None:
        pass
    
    def stylized_name(self) -> str:
        return f"{self.colour} {self.name_prefix} {self.name} {normal}"
    
    def log(self, message: str, display_message = True) -> None:
        log_message = f"({get_stylized_time()})  {self.stylized_name()}: {message}\n"

        with open(file_logs, "a", encoding = 'utf-8') as file:
            file.write(remove_colour(log_message))

        if display_message:
            self.display_message(log_message)

    def log_error(self, error_type, error_message: str) -> None:
        self.log(error_message, False)
        raise error_type(error_message)
    
    def display_message(self, message: str) -> None:
        print(message)

In [7]:
class Habit(Common_Functions):
    def __init__(self, name: str, weight: float, repeat_days: list | tuple) -> None:
        self.name = name
        self.weight = weight
        self.category = None
        self.repeat_days = repeat_days
        
        self.name_prefix = "#"
        self.colour = blue

        self.set_name(name, False)
        self.set_weight(weight, False)
        self.set_repeat_days(repeat_days, False)

        super().__init__()

        self.log(f"Made {self.stylized_name()}!")

    def set_name(self, name: str, record_log = True) -> None:
        raise_error = False
        if not (self.category is None):
            raise_error = self.category.habit_exists(name)

        if raise_error:
            self.log_error(ValueError, f"A Habit with the name \"{name}\" already exists!")

        self.name = name

        if record_log:
            self.log(f"Modified {self.stylized_name()}'s name!")

    def set_weight(self, weight: float, record_log = True) -> None:
        if weight > weight_limit:
            self.log_error(ValueError, f"Weights can't have a value greater than {weight_limit}!")
        
        self.weight = weight

        if record_log:
            self.log(f"Modified {self.stylized_name()}'s weight!")

    def set_category(self, category):
        self.category = category

    def set_repeat_days(self, selected_days: list | tuple, record_log = True) -> None:
        selected_days = [days[index - 1] for index in selected_days]
        self.repeat_days = selected_days

        self.log(f"Successfully modified the repeat days!", record_log)

In [8]:
class Habit_List(Common_Functions):
    def __init__(self, name: str) -> None:
        self.name = name
        self.habits = []

        self.name_prefix = ">"
        self.colour = green

        super().__init__()

        self.log(f"Made {self.stylized_name()}!")

    def net_weight(self) -> float:
        return sum(list(map(lambda habit: habit.weight, self.habits)))
    
    def display_habits(self) -> None:
        print(self.stylized_name())

        for index, habit in enumerate(self.habits):
            print(f"{display_indentation}{index + 1}. {habit.stylized_name()}")

    def habit_exists(self, name: str):
        exists = False
        if self.name == name:
                exists = True

        for habit in self.habits:
            if habit.name == name:
                exists = True
                break

        return exists
    
    @dispatch(Habit)
    def add_habit(self, habit: Habit) -> None:
        if self.habit_exists(habit.name):
            self.log_error(ValueError, f"A Habit with the name \"{habit.name}\" already exists!")

        self.habits.append(habit)
        habit.set_category(self)

        self.log(f"Added {habit.stylized_name()} Successfully!")

    @dispatch(str, int, tuple)
    def add_habit(self, name: str, weight: int, repeat_days: tuple) -> None:
        self.add_habit(name, float(weight), list(repeat_days))

    @dispatch(str, float, tuple)
    def add_habit(self, name: str, weight: int, repeat_days: tuple) -> None:
        self.add_habit(name, weight, list(repeat_days))

    @dispatch(str, int, list)
    def add_habit(self, name: str, weight: int, repeat_days: list) -> None:
        self.add_habit(name, float(weight), repeat_days)

    @dispatch(str, float, list)
    def add_habit(self, name: str, weight: float, repeat_days: list) -> None:
        new_habit = Habit(name, weight, repeat_days)
        new_habit.set_category(self)

        self.habits.append(new_habit)
        self.log(f"Added {new_habit.stylized_name()} Successfully!")

    def delete_habit(self, name: str):
        required_habit = self.get_habit_from_name(name)
        self.habits.remove(required_habit)
        self.log(f"Deleted {required_habit.stylized_name()} Successfully!")

    def get_habit_from_name(self, name: str):
        required_habit = None
        for habit in self.habits:
            if habit.name == name:
                required_habit = habit
                break

        if required_habit is None:
            self.log_error(ValueError, f"No habit with the name \"{name}\" exists!")
        
        self.log(f"Extracted {required_habit.stylized_name()} Successfully!", False)
        return required_habit
    
    @dispatch(int, int)
    def swap(self, habit_1_index: int, habit_2_index: int) -> None:
        self.swap([habit_1_index, habit_2_index])

    @dispatch(tuple)
    def swap(self, swap_positions: tuple) -> None:
        self.swap(list(swap_positions))
    
    @dispatch(list)
    def swap(self, swap_positions: list) -> None:
        swap_positions = list(map(lambda position: position - 1, swap_positions))

        habit_1 = self.habits[swap_positions[0]]
        habit_2 = self.habits[swap_positions[1]]

        self.habits[swap_positions[0]], self.habits[swap_positions[1]] = habit_2, habit_1
        self.log(f"Successfully swaped the order of {habit_1.stylized_name()} and {habit_2.stylized_name()}!")
    
    @dispatch(str, int)
    def change_position(self, habit_name: str, to_index: int) -> None:
        habit = self.get_habit_from_name(habit_name)
        self.change_position(self.habits.index(habit) + 1, to_index)

    @dispatch(int, int)
    def change_position(self, habit_index: int, to_index: int) -> None:
        habit = self.habits.pop(habit_index - 1)
        self.habits.insert(to_index - 1, habit)

        self.log(f"Successfully shifted {habit.stylized_name()} to Position {to_index}!")

    def export_habits(self) -> None:
        with open(file_habits_info, "w", encoding = 'utf-8') as file:
            contents = []
            for habit in self.habits:
                export_format = list(map(str, [habit.category.name, habit.name, habit.weight]))
                contents.append(",".join(export_format) + "\n")

            file.writelines(contents)

In [9]:
playing = Habit_List("Playing 🏃‍♂️")

playing.add_habit("Table Tennis 🏓", 2, [1,3,5,7])
playing.add_habit("Basketball 🏀", 3, [2,6])
playing.add_habit("Chess ♟️", 3.5, [1,2,3,4,5])

bdmton = Habit("Badminton 🏸", 2.5, [4,6,7])
playing.add_habit(bdmton)

(08:03:43 PM  22-Jul-2024)   > Playing 🏃‍♂️ : Made  > Playing 🏃‍♂️ !

(08:03:43 PM  22-Jul-2024)   # Table Tennis 🏓 : Made  # Table Tennis 🏓 !

(08:03:43 PM  22-Jul-2024)   > Playing 🏃‍♂️ : Added  # Table Tennis 🏓  Successfully!

(08:03:43 PM  22-Jul-2024)   # Basketball 🏀 : Made  # Basketball 🏀 !

(08:03:43 PM  22-Jul-2024)   > Playing 🏃‍♂️ : Added  # Basketball 🏀  Successfully!

(08:03:43 PM  22-Jul-2024)   # Chess ♟️ : Made  # Chess ♟️ !

(08:03:43 PM  22-Jul-2024)   > Playing 🏃‍♂️ : Added  # Chess ♟️  Successfully!

(08:03:43 PM  22-Jul-2024)   # Badminton 🏸 : Made  # Badminton 🏸 !

(08:03:43 PM  22-Jul-2024)   > Playing 🏃‍♂️ : Added  # Badminton 🏸  Successfully!



In [10]:
playing.display_habits()

 > Playing 🏃‍♂️ 
   1.  # Table Tennis 🏓 
   2.  # Basketball 🏀 
   3.  # Chess ♟️ 
   4.  # Badminton 🏸 


In [11]:
playing.export_habits()

In [12]:
class Habit_Tracker(Common_Functions):
    def __init__(self, category: Habit_List) -> None:
        self.name = "Habit Tracker 📅"
        self.habits_today = {}
        self.category = category

        self.name_prefix = "~"
        self.colour = orange

    def get_habit_from_name(self, name: str):
        required_habit = None
        for habit in list(self.habits_today.keys()):
            if habit.name == name:
                required_habit = habit
                break

        if required_habit is None:
            self.log_error(ValueError, f"No habit with the name \"{name}\" is present in today's habits!")
        
        self.log(f"Extracted {required_habit.stylized_name()} Successfully!", False)
        return required_habit

    def generate_today(self) -> None:
        day = time.strftime("%a")
        habits_today = {}

        for habit in self.category.habits:
            if day in habit.repeat_days:
                habits_today.update({habit : 0})

        self.habits_today = habits_today
        self.log(f"Successfully generated today's habits!")

    def completed_habit(self, name: str) -> None:
        habit = self.get_habit_from_name(name)
        self.habits_today[habit] = 1

        self.log(f"Completed {habit.stylized_name()}!")
        
    
    def progress(self) -> float:
        progress = 0
        softmax_denominator = 0

        for habit in list(self.habits_today.keys()):
            softmax_numerator = math.e ** habit.weight
            habit_status = self.habits_today[habit]

            progress += softmax_numerator * habit_status
            softmax_denominator += softmax_numerator

        return float(f"{((progress / softmax_denominator) * 100): 0.2f}")
    
    def display_habits_today(self) -> None:
        print(f"{self.stylized_name()} Today {self.name_prefix}")
        for index, habit in enumerate(list(self.habits_today.keys())):
            print(f"{display_indentation}{index + 1}. {habit.stylized_name()} - {habit.weight} - {self.habits_today[habit]}")

        print(f"\n{time_colour}Progress:{normal} {self.progress()}")

In [13]:
habit_track = Habit_Tracker(playing)
habit_track.generate_today()

(08:03:45 PM  22-Jul-2024)   ~ Habit Tracker 📅 : Successfully generated today's habits!



In [14]:
habit_track.completed_habit("Chess ♟️")

(08:03:45 PM  22-Jul-2024)   ~ Habit Tracker 📅 : Completed  # Chess ♟️ !



In [15]:
habit_track.display_habits_today()

 ~ Habit Tracker 📅  Today ~
   1.  # Table Tennis 🏓  - 2.0 - 0
   2.  # Chess ♟️  - 3.5 - 1

Progress: 81.76


In [16]:
habit_track.progress()

81.76

In [17]:
# Goal - Track dates with respect to their habits

with open(file_ID_storage, "a") as file:
    file.
    extracted_IDs = file.readlines()
    used_IDs = list(map(lambda id: id.removesuffix("\n"), extracted_IDs))

print(used_IDs)

while True:
    generated_ID = round((random.random() * (10 ** ID_length)), 0)
    if not (generated_ID in used_IDs):
        break

print(generated_ID)

UnsupportedOperation: not readable